<a href="https://colab.research.google.com/github/maliksh7/CS4104-Applied-Machine-Learning/blob/main/DeepMAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DeepMAD - Malicious Network Activity Detection using Deep Learning**

> ## Member Names: 
    Muhammad Saad Hassan, Mubariz Ahmad Khan 

> ## Roll Numbers: 
    P176137, P180010



In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import packages and libraries
import pandas as pd
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import (
    BatchNormalization, Input,Dense,Flatten,Dropout,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
)

from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers

from keras import backend as K

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [ ]:
# reading dataset from google drive
new_df = pd.read_hdf('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/all_data.h5')

In [ ]:
# info() will tell us the columns/ features and their respective Datatypes 
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2830743 entries, 0 to 2830742
Data columns (total 80 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1    Destination Port             int64  
 2    Flow Duration                int64  
 3    Total Fwd Packets            int64  
 4    Total Backward Packets       int64  
 5   Total Length of Fwd Packets   int64  
 6    Total Length of Bwd Packets  int64  
 7    Fwd Packet Length Max        int64  
 8    Fwd Packet Length Min        int64  
 9    Fwd Packet Length Mean       float64
 10   Fwd Packet Length Std        float64
 11  Bwd Packet Length Max         int64  
 12   Bwd Packet Length Min        int64  
 13   Bwd Packet Length Mean       float64
 14   Bwd Packet Length Std        float64
 15  Flow Bytes/s                  float64
 16   Flow Packets/s               float64
 17   Flow IAT Mean                float64
 18   Flow IAT Std         

In [ ]:
# Replacing inf, - inf with nan values
new_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Dropping all the rows with nan values
new_df.dropna(inplace=True)

In [ ]:
tag = new_df.groupby(' Label')[' Label'].agg('count')
tag

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y=le.fit_transform(new_df[' Label'])
# len(Y)
X=new_df.iloc[:,:-1]

In [ ]:
X=new_df.iloc[:,:-1]

print(X.shape)
X.dropna()
print(X.shape,np.max(np.max(X).astype(np.float32)))
count = -1
for i in np.max(X).astype(np.float32):
    count += 1
    print(count, '---',i)

(2827876, 79)
(2827876, 79) 2071000064.0
0 --- 692702.0
1 --- 65535.0
2 --- 120000000.0
3 --- 219759.0
4 --- 291922.0
5 --- 12900000.0
6 --- 655453056.0
7 --- 24820.0
8 --- 2325.0
9 --- 5940.85693359375
10 --- 7125.5966796875
11 --- 19530.0
12 --- 2896.0
13 --- 5800.5
14 --- 8194.66015625
15 --- 2071000064.0
16 --- 4000000.0
17 --- 120000000.0
18 --- 84800264.0
19 --- 120000000.0
20 --- 120000000.0
21 --- 120000000.0
22 --- 120000000.0
23 --- 84602928.0
24 --- 120000000.0
25 --- 120000000.0
26 --- 120000000.0
27 --- 120000000.0
28 --- 84418016.0
29 --- 120000000.0
30 --- 120000000.0
31 --- 1.0
32 --- 0.0
33 --- 1.0
34 --- 0.0
35 --- 4644908.0
36 --- 5838440.0
37 --- 3000000.0
38 --- 2000000.0
39 --- 1448.0
40 --- 24820.0
41 --- 3337.142822265625
42 --- 4731.5224609375
43 --- 22400000.0
44 --- 1.0
45 --- 1.0
46 --- 1.0
47 --- 1.0
48 --- 1.0
49 --- 1.0
50 --- 1.0
51 --- 1.0
52 --- 156.0
53 --- 3893.333251953125
54 --- 5940.85693359375
55 --- 5800.5
56 --- 4644908.0
57 --- 0.0
58 --- 0.0


In [ ]:
np.random.seed(0)
# Train Test split (train-> 80%, test-> 20%)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
print(x_train.shape,len(y_train),x_test.shape,len(y_test))
x_train.columns

(2262300, 79) 2262300 (565576, 79) 565576


Index(['Unnamed: 0', ' Destination Port', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Lengt

In [ ]:
# removing extra/ unnecessary columns/ features
del new_df['Unnamed: 0']
del new_df['Fwd Avg Bytes/Bulk']
del new_df[' SYN Flag Count']
del new_df['Bwd Avg Bulk Rate']
del new_df[' RST Flag Count']
del new_df[' Fwd Avg Packets/Bulk']
del new_df[' Fwd Avg Bulk Rate']
del new_df[' ECE Flag Count']
del new_df[' CWE Flag Count']
del new_df[' Bwd URG Flags']
del new_df[' Bwd PSH Flags']
del new_df[' Bwd Avg Packets/Bulk']
del new_df[' Bwd Avg Bytes/Bulk']
del new_df[' Bwd Packets/s']
del new_df['Active Mean']
del new_df[' Active Min']
del new_df[' Total Length of Bwd Packets']
del new_df[ ' Bwd IAT Std']
del new_df[ ' Bwd IAT Mean']
del new_df[' Active Std']
del new_df[ ' Bwd IAT Min']
del new_df[' Down/Up Ratio']
del new_df[' Active Max']
del new_df[' Fwd URG Flags']


In [ ]:
# checking multi-class label values(attacks/benign)
new_df[' Label'].value_counts()

BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name:  Label, dtype: int64

In [ ]:
# applying label encoder as le
new_df[' Label'] = le.fit_transform(new_df[' Label'])

In [ ]:
new_df[' Label'].value_counts()

0     2271320
4      230124
10     158804
2      128025
3       10293
7        7935
11       5897
6        5796
5        5499
1        1956
12       1507
14        652
9          36
13         21
8          11
Name:  Label, dtype: int64

In [ ]:
# Locate the value of multiclass label and select those with huge size (Label -> 0, 2, 4, 10)

benign_df = new_df.loc[new_df[' Label'] == 0]
print(benign_df.shape)

DDos_df = new_df.loc[new_df[' Label'] == 2]
print(DDos_df.shape)

dos_hulk_df = new_df.loc[new_df[' Label'] == 4]
print(dos_hulk_df.shape)

portscan_df = new_df.loc[new_df[' Label'] == 10]
print(portscan_df.shape)

(2271320, 56)
(128025, 56)
(230124, 56)
(158804, 56)


In [ ]:
# concat all the selected dataframes
frames = [benign_df,DDos_df,dos_hulk_df,portscan_df]
result = pd.concat(frames)

In [ ]:
# Dropping all the rows with nan values
result.replace([np.inf, -np.inf], np.nan, inplace=True)
result.dropna(inplace=True)

In [ ]:
# saving selected dataframes in HDF5 Format
filename = ('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/4_Atack_pure_data.h5')
result.to_hdf(filename, 'data', mode='w', format='table')

In [ ]:
# shape of selected dataframe
print(result.shape)
print(2271320+128025+230124+158804)


(2788273, 56)
2788273


In [ ]:
result[' Label'].value_counts()

0     2271320
4      230124
10     158804
2      128025
Name:  Label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y=le.fit_transform(result[' Label'])
len(Y)
X=result.iloc[:,:-1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
print(x_train.shape,len(y_train),x_test.shape,len(y_test))
len(x_train.columns)

(2230618, 55) 2230618 (557655, 55) 557655


55

## Decision Tree Classifier

In [ ]:
print(x_train.shape, y_train.shape)
dtc = DecisionTreeClassifier(max_depth = 15)
dtc = dtc.fit(x_train,y_train)

y_pred = dtc.predict(x_test)
y_pred

from sklearn.metrics import accuracy_score,f1_score
print(dtc.score(x_test, y_test))
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average='macro'))

(2230618, 55) (2230618,)
0.9993006428705921
0.9993006428705921
0.9985072223035766


In [ ]:
validation_accuracy = []
for depth in range(1,15):
    dt_model = DecisionTreeClassifier(max_depth=depth, random_state=10)
    dt_model.fit(x_train, y_train)
    train_accuracy.append(dt_model.score(x_train, y_train))
    validation_accuracy.append(dt_model.score(x_test, y_test))
print(len(train_accuracy))
print(len(validation_accuracy))
frame = pd.DataFrame({'max_depth':range(1,15), 'train_acc':train_accuracy, 'valid_acc':validation_accuracy})

## Random Forest CLassifier

In [ ]:
print('---------------------------Random Forest CLassifier------------------')
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators =5)
model = model.fit(x_train, y_train)
rfy_pred = model.predict(x_test)
acc = metrics.accuracy_score(y_test, rfy_pred)
print(f1_score(y_test, rfy_pred, average='macro'), 'f1')
print(model.score(x_test,y_test))

---------------------------Random Forest CLassifier------------------
0.9981037692108208 f1
0.9991464256574405


In [ ]:
print(acc)

0.9991464256574405


## Extra Trees Classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(x_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [ ]:
rfy_pred = model.predict(x_test)
acc = metrics.accuracy_score(y_test, rfy_pred)
print(acc)
print(f1_score(y_test, rfy_pred, average='macro'), 'f1')
print(model.score(x_test,y_test))

0.9989545507527056
0.9977493432537416 f1
0.9989545507527056


## Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(x_train, y_train)
rfy_pred = gnb.predict(x_test)


In [ ]:
acc = metrics.accuracy_score(y_test, rfy_pred)
print(f1_score(y_test, rfy_pred, average='macro'), 'f1')
print(model.score(x_test,y_test))

0.350347153996948 f1
0.9989545507527056


## Logistic Regression

In [ ]:
clf_logistic = LogisticRegression()
clf_logistic = clf_logistic.fit(x_train, y_train)
rfy_pred = clf_logistic.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# acc = metrics.accuracy_score(y_test, rfy_pred)
print(f1_score(y_test, rfy_pred, average='macro'), 'f1')
print(clf_logistic.score(x_test,y_test))

0.5940608963857186 f1
0.876486358052918


## Multi-Layered Perceptron (MLP)

In [ ]:
df = pd.read_hdf('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/800k-Dos-DDos-PS-Benign.h5')
df[' Label'].value_counts()

0    500000
3    100000
2    100000
1    100000
Name:  Label, dtype: int64

In [ ]:
df.head(10)

In [ ]:
del df['Unnamed: 0']
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Max',
       'Fwd PSH Flags', ' Fwd Header Length', ' Bwd Header Length',
       'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' PSH Flag Count', ' ACK Flag Count',
       ' URG Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size',
       ' Avg Bwd Segment Size', ' Fwd

In [ ]:
from sklearn.preprocessing import LabelEncoder

df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Dropping all the rows with nan values
df.dropna(inplace=True)

# le = LabelEncoder()
Y=df[' Label']
# len(Y)
X=df.iloc[:,:-1]

In [ ]:


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.35, random_state = 42)
print(x_train.shape,len(y_train),x_test.shape,len(y_test))
len(x_train.columns)

(520000, 55) 520000 (280000, 55) 280000


55

In [ ]:
df[' Label'].value_counts()

0    500000
3    100000
2    100000
1    100000
Name:  Label, dtype: int64

In [ ]:
# Imports
import tensorflow
# from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

# Configuration options
feature_vector_length = 55
num_classes = 4

 # Y_train, Y_test

Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)



In [ ]:
num_classes

4

In [ ]:
# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()
# adding input shape
model.add(Input(shape=(55 )))

model.add(Dense(28, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(14, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))


Feature shape: (55,)


In [ ]:
# model.summary() -> will tell model trainable/ non-trainable parameters, droupout and outputshape of each layer.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 28)                1568      
                                                                 
 dropout (Dropout)           (None, 28)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                406       
                                                                 
 dropout_1 (Dropout)         (None, 14)                0         
                                                                 
 dense_2 (Dense)             (None, 7)                 105       
                                                                 
 dropout_2 (Dropout)         (None, 7)                 0         
                                                                 
 dense_3 (Dense)             (None, 4)                 3

In [ ]:
type(x_train)

pandas.core.frame.DataFrame

In [ ]:
# Configure the model and start training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
model.fit(x_train, Y_train, epochs=100, batch_size=64, verbose=1, validation_data=(x_test, Y_test))

# Test the model after training
test_results = model.evaluate(x_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

Epoch 1/100
8125/8125 [==============================] - 20s 2ms/step - loss: 0.1034 - accuracy: 0.9702 - mse: 0.0116 - val_loss: 0.0392 - val_accuracy: 0.9916 - val_mse: 0.0034
Epoch 2/100
8125/8125 [==============================] - 20s 2ms/step - loss: 0.0932 - accuracy: 0.9732 - mse: 0.0104 - val_loss: 0.0365 - val_accuracy: 0.9927 - val_mse: 0.0031
Epoch 3/100
8125/8125 [==============================] - 22s 3ms/step - loss: 0.0871 - accuracy: 0.9750 - mse: 0.0098 - val_loss: 0.0503 - val_accuracy: 0.9939 - val_mse: 0.0026
Epoch 4/100
8125/8125 [==============================] - 21s 3ms/step - loss: 0.0865 - accuracy: 0.9752 - mse: 0.0097 - val_loss: 0.0291 - val_accuracy: 0.9934 - val_mse: 0.0026
Epoch 5/100
8125/8125 [==============================] - 22s 3ms/step - loss: 0.0808 - accuracy: 0.9774 - mse: 0.0089 - val_loss: 0.0250 - val_accuracy: 0.9935 - val_mse: 0.0026
Epoch 6/100
8125/8125 [==============================] - 20s 2ms/step - loss: 0.0779 - accuracy: 0.9782 - mse:

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520000 entries, 2253029 to 250032
Data columns (total 55 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0    Destination Port            520000 non-null  uint8
 1    Flow Duration               520000 non-null  uint8
 2    Total Fwd Packets           520000 non-null  uint8
 3    Total Backward Packets      520000 non-null  uint8
 4   Total Length of Fwd Packets  520000 non-null  uint8
 5    Fwd Packet Length Max       520000 non-null  uint8
 6    Fwd Packet Length Min       520000 non-null  uint8
 7    Fwd Packet Length Mean      520000 non-null  uint8
 8    Fwd Packet Length Std       520000 non-null  uint8
 9   Bwd Packet Length Max        520000 non-null  uint8
 10   Bwd Packet Length Min       520000 non-null  uint8
 11   Bwd Packet Length Mean      520000 non-null  uint8
 12   Bwd Packet Length Std       520000 non-null  uint8
 13  Flow Bytes/s           

In [ ]:
new_df[' Label'].value_counts()

In [ ]:
test_results = model.evaluate(x_test, Y_test, verbose=1)

8750/8750 [==============================] - 15s 2ms/step - loss: 0.0222 - accuracy: 0.9941 - mse: 0.0021


In [ ]:
x_test.head(10)

In [ ]:
l = model.predict(x_test, verbose=0)

In [ ]:
l = np.argmax(l, axis=1)
prediction_int = np.zeroes_like(prediction)
prediction_int[prediction > 0.5] = 1

In [ ]:
print(l)
# rounding off values of l
l = l.round()
print(l)

[[9.9999988e-01 9.4183157e-08 4.1691074e-11 0.0000000e+00]
 [1.0000000e+00 6.2043266e-11 4.0033752e-16 0.0000000e+00]
 [1.0000000e+00 1.6741014e-10 1.0743874e-15 0.0000000e+00]
 ...
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]
 [1.0000000e+00 5.3965986e-21 3.7108573e-32 0.0000000e+00]
 [1.8769801e-05 7.1809183e-17 9.9998128e-01 0.0000000e+00]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [ ]:
from sklearn.metrics import f1_score


In [ ]:
print(f1_score(Y_test, l, average='macro'), 'f1') 
# 0.22442756164188604 f1


0.992935546000768 f1


In [ ]:
l

array([[0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       ...,
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ]], dtype=float32)

In [ ]:
Y_test

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [ ]:
filename = ('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/800kValidation-Dos-DDos-PS-Benign.h5')
valid_df = pd.read_hdf(filename)

In [ ]:
del valid_df['Unnamed: 0']
valid_df.columns

In [ ]:
valid_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Dropping all the rows with nan values
valid_df.dropna(inplace=True)

# le = LabelEncoder()
Y=valid_df[' Label']
# len(Y)
X=valid_df.iloc[:,:-1]

In [ ]:
# Y_train = to_categorical(, num_classes)
Y_test = to_categorical(Y, num_classes)

In [ ]:
# model predict
check = model.predict(X, verbose=0)

In [ ]:
print(check)
check = check.round()
print(check)

[[1.0000000e+00 1.3289315e-13 2.5395361e-20 0.0000000e+00]
 [1.0000000e+00 8.0662291e-12 2.2246202e-17 0.0000000e+00]
 [1.0000000e+00 3.3879440e-21 9.8526837e-35 0.0000000e+00]
 ...
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [ ]:
print(f1_score(Y_test, check, average='macro'), 'f1') 
# 0.22442756164188604 f1


0.99187354905695 f1


In [ ]:
# saving model weights
filename = ('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/MNAD-DNN.h5')
# model.save_weights(filename)

In [ ]:
filename = ('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/MNAD-DNN.h5')
